In [4]:
import torch
import onnx
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnx
import onnxruntime as rt
from torchviz import make_dot
import netron
from modeci_mdf.interfaces.pytorch import pytorch_to_mdf
import torch
import torchvision.models as models


from onnx import helper

from modeci_mdf.interfaces.onnx import onnx_to_mdf, convert_file




from modeci_mdf.interfaces.onnx import onnx_to_mdf

In [5]:
model1 = models.resnet18()
model1.load_state_dict(torch.load('/content/resnet18.pth'))

<All keys matched successfully>

In [7]:
def main():

    model =model1
    input_images = torch.zeros((1, 3, 224, 224), requires_grad=False)
    model.eval()
    # Run the model once to get some ground truth outpot (from PyTorch)
    
    # Seed the random number generator to get deterministic behavior for weight initialization
    torch.manual_seed(0)
    output = model(input_images)
    print(f"PyTorch model: {model} created. Evaluates {input_images} as {output}")

    torch.onnx.export(
        model,
        (input_images),
        "pt.onnx",
        verbose=True,
        input_names=["input"],
        opset_version=9,
    )

    # Load it back in using ONNX package
    onnx_model = onnx.load("pt.onnx")
    onnx.checker.check_model(onnx_model)

    import onnxruntime as rt

    sess = rt.InferenceSession("pt.onnx")

    res = sess.run(
        [sess.get_outputs()[0].name], {sess.get_inputs()[0].name: input_images.numpy()}
    )
    print(f"Output calculated by onnxruntime (input: {input_images}):  {res}")

    mdf_model = onnx_to_mdf(onnx_model)
    mdf_model.to_json_file("pt.json")
    mdf_model.to_yaml_file("pt.yaml")

    mdf_model.to_graph_image(
        engine="dot",
        output_format="png",
        view_on_render=False,
        level=3,
        filename_root="pt",
        only_warn_on_fail=True,  # Makes sure test of this doesn't fail on Windows on GitHub Actions
    )
    if "-run" in sys.argv:
        verbose = True
        verbose = False

        from modeci_mdf.execution_engine import EvaluableGraph

        eg = EvaluableGraph(mdf_model.graphs[0], verbose=verbose)

        print("Evaluating graph...")
        test_values = [0, 1, [1, 2], input_images.numpy()]
        test_values = [0, 1, [1, 2]]

        for t in test_values:
            print("===================\nEvaluating MDF model with input: %s" % t)
            eg.evaluate(initializer={"input": t})
            print("Output: %s" % eg.enodes["Mul_3"].evaluable_outputs["_4"].curr_value)


if __name__ == "__main__":
    main()


PyTorch model: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(i

In [8]:
def main():

    model = model1
    input_images = torch.zeros((1, 3, 224, 224), requires_grad=False)
    model.eval()
    # Run the model once to get some ground truth outpot (from PyTorch)
    
    # Seed the random number generator to get deterministic behavior for weight initialization
    torch.manual_seed(0)
    output = model(input_images)
    # loop_count = torch.tensor(5, dtype=torch.long)
    torch.onnx.export(
        model, (input_images), "pyt_basic.onnx", verbose=True, input_names=["input"]
    )

    # Load it back in using ONNX package
    onnx_model = onnx.load("pyt_basic.onnx")
    print(onnx_model)
    onnx.checker.check_model(onnx_model)

    # Extract the loop or if body as a sub-model, this is just because I want
    # to view it in netron and sub-graphs can't be rendered
    for node in [
        node for node in onnx_model.graph.node if node.op_type in ["Loop", "If"]
    ]:

        # Get the GraphProto of the body
        body_graph = node.attribute[0].g

        # Turn it into a model
        model_def = helper.make_model(body_graph, producer_name="pyt_basic.py")

        onnx.save(model_def, f"examples/{node.name}_body.onnx")

    convert_file("pyt_basic.onnx")


if __name__ == "__main__":
    main()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
def main():

  
    model = model1

    model = torch.jit.script(model)

    

    

    input_images = torch.zeros((1, 3, 224, 224), requires_grad=False)
    output = model(input_images)
    print(output)
    torch.onnx.export(
        model,
        (input_images),
        "pytc.onnx",
        verbose=True,
        input_names=["input"],
        opset_version=9,
    )

    # Load it back in using ONNX package
    onnx_model = onnx.load("pytc.onnx")
    onnx.checker.check_model(onnx_model)

    mdf_model = onnx_to_mdf(onnx_model)

    mdf_model.to_json_file("pytc.json")
    mdf_model.to_yaml_file("pytc.yaml")


if __name__ == "__main__":
    main()

tensor([[-2.8774e-02, -5.2932e-03, -3.6097e-02,  6.8579e-03, -3.4067e-03,
         -3.0889e-02,  2.4735e-02,  4.2477e-02,  2.6443e-03,  2.1936e-02,
         -3.2019e-02,  3.1720e-02, -4.2142e-02, -2.9292e-02,  1.8174e-02,
         -1.4973e-02,  2.7067e-02, -9.0325e-03,  2.1031e-02, -9.1640e-03,
         -6.0370e-03, -1.6253e-02, -4.0052e-02,  3.8943e-02,  3.0394e-02,
         -1.6035e-02,  3.3327e-02,  1.1011e-02,  2.5651e-02,  6.3637e-03,
         -4.0436e-02, -3.2286e-02, -5.7262e-03, -3.6450e-02, -3.5967e-02,
         -2.9956e-02, -3.4702e-04,  1.7811e-02,  5.9253e-03,  3.3936e-03,
          1.7460e-02,  4.2890e-02,  3.0843e-02,  4.2885e-02,  1.4251e-02,
         -3.3574e-02,  2.6518e-02, -4.1100e-02, -3.5351e-02,  4.1666e-02,
          3.2555e-02,  1.9023e-03, -3.2874e-02,  5.3508e-03, -1.6104e-02,
          2.0862e-02,  4.0813e-02, -8.4683e-03,  1.7769e-02, -1.7465e-02,
         -1.0925e-04,  6.1013e-03, -2.1344e-03, -2.1584e-02,  7.4184e-05,
          1.4014e-03,  3.5097e-02,  1.